## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-19-00-02 +0000,wapo,Walz defends Minnesota immigration enforcement...,https://www.washingtonpost.com
1,2026-01-26-19-00-02 +0000,wapo,Live updates: Court holding hearings in Minnes...,https://www.washingtonpost.com/nation/2026/01/...
2,2026-01-26-18-59-55 +0000,nypost,NYC mom charged with allegedly stuffing body o...,https://nypost.com/2026/01/26/us-news/nyc-mom-...
3,2026-01-26-18-58-11 +0000,bbc,Minister claims 'sabotage' behind Chagos debat...,https://www.bbc.com/news/articles/czjgwllmy99o...
4,2026-01-26-18-58-06 +0000,nypost,Beverly Hills theater apologizes after canceli...,https://nypost.com/2026/01/26/us-news/beverly-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2468/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
68,trump,41
2,minnesota,25
170,shooting,22
188,minneapolis,20
88,storm,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
243,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,141
159,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...,138
162,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,112
135,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...,105
61,2026-01-26-17-16-14 +0000,startribune,Live: Trump speaks with Walz as judge holds he...,https://www.startribune.com/ice-raids-minnesot...,102


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
243,141,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
135,64,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...
53,63,2026-01-26-17-34-38 +0000,wapo,Seven killed in private jet crash at Maine air...,https://www.washingtonpost.com/transportation/...
210,45,2026-01-26-08-36-39 +0000,wsj,Stock Market Today: Dow Rises; Gold Price Tops...,https://www.wsj.com/livecoverage/stock-market-...
134,44,2026-01-26-14-25-51 +0000,nypost,Final Israeli hostage in Gaza finally returned...,https://nypost.com/2026/01/26/world-news/body-...
265,41,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
182,36,2026-01-26-11-00-00 +0000,startribune,Viewing violent ICE videos takes a toll. Here’...,https://www.startribune.com/videos-show-ice-ar...
42,34,2026-01-26-17-54-24 +0000,nypost,At least 18 dead after ferry carrying over 350...,https://nypost.com/2026/01/26/world-news/at-le...
0,29,2026-01-26-19-00-02 +0000,wapo,Walz defends Minnesota immigration enforcement...,https://www.washingtonpost.com
32,29,2026-01-26-18-13-00 +0000,wsj,GOP candidate Chris Madel is dropping out of t...,https://www.wsj.com/politics/policy/the-lawyer...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
